In [1]:
import BAC0

In [2]:
bacnet = BAC0.connect(port=47811)

2024-12-30 10:17:30,065 - INFO    | Starting BAC0 version 23.07.03 (Lite)
2024-12-30 10:17:30,066 - INFO    | Use BAC0.log_level to adjust verbosity of the app.
2024-12-30 10:17:30,066 - INFO    | Ex. BAC0.log_level('silence') or BAC0.log_level('error')
2024-12-30 10:17:30,067 - INFO    | Starting TaskManager
2024-12-30 10:17:30,077 - INFO    | Using ip : 10.84.0.66:47811 on port 47811 | broadcast : 10.84.0.255
2024-12-30 10:17:30,117 - INFO    | Starting app...
2024-12-30 10:17:30,118 - INFO    | BAC0 started
2024-12-30 10:17:30,119 - INFO    | Registered as Simple BACnet/IP App
2024-12-30 10:17:30,119 - INFO    | Device instance (id) : 3056438
2024-12-30 10:17:30,150 - INFO    | Update Local COV Task started (required to support COV)


In [3]:
def leer_punto(ip,tipo,direccion):
    try:
        return bacnet.read(f'{ip} {tipo} {direccion} presentValue')
    except:
        return ''
"""
def descubrir_puntos(ip):
    tipos = ["binaryOutput",
    "binaryInput",
    "multiStateOutput",
    "multiStateInput",
    "analogValue",
    "analogInput",
    "analogOutput"
    ]
    for direccion in range(0,)
    """
        

'\ndef descubrir_puntos(ip):\n    tipos = ["binaryOutput",\n    "binaryInput",\n    "multiStateOutput",\n    "multiStateInput",\n    "analogValue",\n    "analogInput",\n    "analogOutput"\n    ]\n    for direccion in range(0,)\n    '

In [5]:
# leer_punto("10.84.37.185", "binaryOutput", "6401")
bacnet.read(f'10.84.37.185 binaryOutput 6401 presentValue')

NoResponseFromController: APDU Abort Reason : Timeout

In [44]:
tabla = bacnet.readMultiple("10.84.67.185 device 1001 all")

In [45]:
puntos = tabla[-15]
tabla[-15]

[('device', 9001),
 ('binaryOutput', 257),
 ('binaryInput', 258),
 ('binaryOutput', 259),
 ('binaryInput', 260),
 ('binaryInput', 261),
 ('binaryValue', 262),
 ('multiStateOutput', 263),
 ('multiStateInput', 264),
 ('binaryOutput', 265),
 ('binaryInput', 266),
 ('multiStateOutput', 267),
 ('multiStateInput', 268),
 ('analogValue', 269),
 ('analogInput', 270),
 ('binaryInput', 271),
 ('analogInput', 272),
 ('multiStateOutput', 273),
 ('multiStateInput', 274),
 ('analogInput', 275),
 ('analogInput', 276),
 ('multiStateOutput', 277),
 ('multiStateInput', 278),
 ('binaryOutput', 279),
 ('binaryInput', 280),
 ('binaryOutput', 281),
 ('binaryInput', 282),
 ('analogValue', 283),
 ('analogValue', 284),
 ('analogInput', 285),
 ('analogInput', 286),
 ('binaryOutput', 287),
 ('binaryInput', 288),
 ('binaryOutput', 289),
 ('binaryInput', 290),
 ('binaryOutput', 291),
 ('binaryInput', 292),
 ('analogValue', 293),
 ('analogInput', 294),
 ('analogValue', 295),
 ('analogInput', 296),
 ('analogValue', 

In [32]:
import json
mapeo_modbus = []
coil_offset = 0
register_offset = 0

for tipo, direccion in puntos:
    if tipo in ["binaryInput", "binaryOutput"]:
        mapeo_modbus.append({
            "bacnet_tipo": tipo,
            "bacnet_direccion": direccion,
            "modbus_tipo": "Coil",
            "modbus_direccion": coil_offset,
        })
        coil_offset += 1
    elif tipo in ["analogInput", "analogOutput", "analogValue"]:
        mapeo_modbus.append({
            "bacnet_tipo": tipo,
            "bacnet_direccion": direccion,
            "modbus_tipo": "Holding Register",
            "modbus_direccion": register_offset,
        })
        register_offset += 1

with open("archivo.json", "w") as archivo:
    json.dump(mapeo_modbus, archivo, indent=4)

In [6]:
from pymodbus.client import ModbusTcpClient

# Configuración del cliente
client = ModbusTcpClient('10.84.67.18', port=502)

# Conexión
if client.connect():
    # Lectura de 10 registros de retención desde la dirección 0
    response = client.read_holding_registers(address=0, count=10, unit=1)

    if not response.isError():  # Verifica si no hubo errores
        print("Registros:", response.registers)
    else:
        print("Error al leer registros:", response)

    # Cerrar conexión
    client.close()
else:
    print("No se pudo conectar al dispositivo Modbus.")


Connection to (10.84.67.18, 502) failed: timed out


No se pudo conectar al dispositivo Modbus.


In [11]:
import pymodbus
from pymodbus.client import ModbusTcpClient

In [15]:
# Crear una conexión con el dispositivo Modbus
client = pymodbus.client.ModbusTcpClient("10.84.67.185", 502)

# Leer el valor del registro de datos
register = 10
response = client.read_holding_registers(register, 1)

# Imprimir el valor del registro de datos
print(response.registers[0])

AttributeError: module 'pymodbus.client' has no attribute 'ModbusClient'

In [14]:
help(ModbusClient)


Help on class ModbusTcpClient in module pymodbus.client.tcp:

class ModbusTcpClient(pymodbus.client.base.ModbusBaseSyncClient)
 |  ModbusTcpClient(host: 'str', *, framer: 'FramerType' = <FramerType.SOCKET: 'socket'>, port: 'int' = 502, name: 'str' = 'comm', source_address: 'tuple[str, int] | None' = None, reconnect_delay: 'float' = 0.1, reconnect_delay_max: 'float' = 300, timeout: 'float' = 3, retries: 'int' = 3, trace_packet: 'Callable[[bool, bytes], bytes] | None' = None, trace_pdu: 'Callable[[bool, ModbusPDU], ModbusPDU] | None' = None, trace_connect: 'Callable[[bool], None] | None' = None) -> 'None'
 |  
 |  **ModbusTcpClient**.
 |  
 |  Fixed parameters:
 |  
 |  :param host: Host IP address or host name
 |  
 |  Optional parameters:
 |  
 |  :param framer: Framer name, default FramerType.SOCKET
 |  :param port: Port used for communication
 |  :param name: Set communication name, used in logging
 |  :param source_address: source address of client
 |  :param reconnect_delay: Not us